# Instructions

_To make the most of the workshop you should bring your laptop with Anaconda installed (see [anaconda](https://www.anaconda.com/products/distribution%3E) for how to install). Once conda is installed you can already download MNE python (https://mne.tools/) to save some time during the set-up, a recommended way is to use a dedicated conda environment as follows (see [conda managing environments](https://conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html#)):_
```
$ conda create --override-channels --channel=conda-forge --name=hmp python=3.11 mne
```

Then activate the created environment through the terminal (linux/MacOS) or through the Anaconda Prompt (windows):
```
conda activate hmp
pip install hsmm_mvpy
```

If everything went without error you can download the tutorials folder on the associated github page https://github.com/GWeindel/hsmm_mvpy/tree/main/tutorials
If you had troubles during the installation let us know!

Navigate to the folder where you downloaded the repository through the terminal/prompt and launch a jupyter notebook session (alternatively you can also first launch the following command and then navigate to the folder):

```
jupyter notebook
```

You should see the content of the folder you downloaded, during the workshop we will be using the notebooks in the folder /Workshop.
After this execute these cells up to data format and wait for the workshop to resume

# Practical, methodological and theoretical grounds

## Data 

### Simulation

We will start by simulating data to demonstrate the HMP analysis on. First, we need to load several libraries, after which we simulate a dataset with 4 events, resulting 5 stages (stimulus -> event 1, event 1 -> event 2, ..., event 4 -> response.

In [ ]:
## Importing these packages is specific for this simulation case
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import gamma

## Importing HMP
import hsmm_mvpy as hmp
from hsmm_mvpy import simulations

In [ ]:
cpus = 2 # For multiprocessing, usually a good idea to use multiple CPUs as long as you have enough RAM

n_trials = 100 #Number of trials to simulate
sfreq = 500
##### Here we define the sources of the brain activity (event) for each trial
n_events = 4
n_stages = n_events+1
frequency = 10. #Frequency of the event defining its duration, half-sine of 10Hz = 50ms
amplitude = .2e-6 #Amplitude of the event in nAm, defining signal to noise ratio
shape = 2 #shape of the gamma distribution
means = np.array([60, 150, 200, 100, 80])/shape #Mean duration of the stages in ms
names = simulations.available_sources()[:n_stages+1] #Which source to activate at each stage (see atlas when calling simulations.available_sources())

sources = []
for source in zip(names, means): #One source = one frequency, one amplitude and a given by-trial variability distribution
    sources.append([source[0], frequency, amplitude, gamma(shape, scale=source[1])])

# Function used to generate the data
file = simulations.simulate(sources, n_trials, cpus, 'dataset_raw', overwrite=False, sfreq=sfreq, seed=1234)
#load electrode position, specific to the simulations
positions = simulations.simulation_positions()


Next, we load the simulated data, reflecting a realistic EEG dataset of a single participant. As you can see, we use standard MNE functions here.

In [ ]:
#Recovering the events to epoch the data (in the number of trials defined above)
events = np.load(file[1])
resp_trigger = int(np.max(np.unique(events[:,2])))#Resp trigger is the last source in each trial
event_id = {'stimulus':1}#trigger 1 = stimulus
resp_id = {'response':resp_trigger}
#Keeping only stimulus and response triggers

#Visualising the raw simulated EEG data
import mne
raw = mne.io.read_raw_fif(file[0], preload=False, verbose=False)
raw.pick_types(eeg=True).plot(scalings=dict(eeg=1e-5), events=events, block=True);

### Data Format 
We now prepare the data for the HMP package. First, we read the data into an xarray.

In [ ]:
# Reading the data
eeg_data = hmp.utils.read_mne_data(file[0], event_id=event_id, resp_id=resp_id, sfreq=sfreq, 
            events_provided=events, verbose=False)
eeg_data

The previous function epochs the data, cut the EEG data at each RT value and stores any additional infomation along (here 'event_name').

xarray is a handy way of using matrices with named dimensions. For example we can now very easily draw an ERP:


In [ ]:
#example of usage of xarray
print(eeg_data)
eeg_data.sel(channels=['EEG 001','EEG 002','EEG 003'], samples=range(400))\
    .data.groupby('samples').mean(['participant','epochs']).plot.line(hue='channels');

Next, we transform the data for HMP:
- Standardise the individual variances to make participants comparable
- Apply PCA and select top components to lower computational load
- zscore the data

This analysis step asks you to choose a number of PC components; 4 is a reasonable answer based on the 'elbow method'.

In [ ]:
hmp_data = hmp.utils.transform_data(eeg_data, apply_standard=False)

In [ ]:
hmp.visu.plot_components_sensor(hmp_data, positions)

## HMP and implementation assumptions

HMP uses classes to instantiate fit of hmp models. Therefore before estimating a model one needs to declare an instance of the hmp class by specifying the data, the sampling frequency of the data as well as several parmeters such as the event width (in milliseconds) and the expected distribution (both family and shape).

By default, we use the same assumptions for the events and stage distributions as in the original paper of Anderson, Zhang, Borst & Walsh, 2016. That is, we use an event template of a half sine wave and gamma distributions with a shape parameter of 2 to model the data. Note that you can very easily use other templates and distributions in HMP.

In the next step, we initialize an hmp model 'init' with those standard settings, using the data we simulated above.

In [ ]:
init = hmp.models.hmp(data=hmp_data, epoch_data=eeg_data, sfreq=eeg_data.sfreq,
                      event_width=50, distribution='gamma', shape=2)

We can view the template used by init: a 50 ms bump (i.e. a 10Hz half-sin):

In [ ]:
plt.plot(init.template, 'x')
plt.ylabel('Normalized value')
plt.xlabel('Samples NOT time');

When calling ```hmp.models.hmp```, the function automatically cross-correlated the data with the provided template, to get an estimate of where the EEG data looks like the template.

Below, we plot those cross-correlated values; the vertical lines indicate the start of the events in the simulated data.

In [ ]:
plt.plot(init.data_matrix[:,0,:]);

#Recover the actual time of the simulated events
sim_source_times = np.reshape(np.ediff1d(events[:,0],to_begin=0)[events[:,2] > 1], (n_trials, n_events+1))
plt.vlines(sim_source_times[0,:-1].cumsum()-1, -3, 3, 'k');#overlaying the simulated stage transition times

As you can see, some of the events are recognizable in the cross-correlations, while others (2 and 3rd) are much harder to detect.

### HMP parameters

An HMP model tries to recover the underlying magnitudes (= EEG topologies) and stage distribution parameters from the data. Here, we first we retrieve the actual generating parameters from the simulation, in order to judge how well the HMP recovery works.

In [ ]:
_, true_pars, true_magnitudes, _ = simulations.simulated_times_and_parameters(events, init)

As an exceedingly simple example, we will first feed these parameters and magnitudes into the HMP model fitting procedure as starting points, and try to estimate a model.

We do that by calling `fit_single` on `init`, and telling it to recover 4 events and using the true parameters and magnitudes as starting points. We also tell it that it does not in fact have to maximize the fit.

In [ ]:
estimates = init.fit_single(n_events, parameters = true_pars, magnitudes=true_magnitudes, maximization=False)

hmp.visu.plot_topo_timecourse(eeg_data, estimates, positions, init, magnify=1, sensors=True, times_to_display = np.mean(np.cumsum(sim_source_times,axis=1),axis=0))

In the plot above we see the four recovered events; the vertical lines again indicate the start of the actual events. Not surprisingly, the recovery is perfect.

The first element that we fed into the fit_single function were the expected distributions for stage duration, `true_pars`. Let's have a look at those:

In [ ]:
colors = sns.color_palette(None, n_stages)

T = 350
for stage in range(5):
    plt.plot(np.linspace(0,T,1001),gamma.pdf(np.linspace(0,T,1001), 2, scale=true_pars[stage,1]), label=f'Stage {stage}', color=colors[stage]) 
plt.xlabel('t')
plt.legend()

The width of these distribution indicates the variability of the stages durations over trials and (with real data) participants.

The second element is the contribution of the different electrodes/magnometers/PCs to each of the event, the `true_mags`. Here we get the estimated magnitudes:

In [ ]:
estimates.magnitudes.dropna('event').T.plot()

We used 4 PC components (y-axis) to estimate 4 events. As is visible in the matrix above, each event has a unique combination of component contributions, although this is not a requirement.

To estimate this set of parameters (magnitudes and gamma scales) we use the expectation maximization algorithm on the event probability as obtained through the Baum-Welsh algorithm. 

Now we do not provide the true magnitudes and parameters as starting points, but start with neutral ones (0's for the magnitudes, mean duration / n_events for the parameters).

In [ ]:
estimates = init.fit_single(n_events, tolerance=1e-8)

hmp.visu.plot_topo_timecourse(eeg_data, estimates, positions, init, magnify=1, sensors=True, times_to_display = np.mean(np.cumsum(sim_source_times,axis=1),axis=0))

Also without providing the true magnitudes and parameters, the algorithm manages to recover the events in the simulated data.

We can inspect the traces of the EM algorithm to see how the log-likelihood of the fit developed with iterations. This shows how many iterations it needed and at what Log-likelihood (LL) it stopped according to the default tolerance on the increase in LL (default 1e-4).

In [ ]:
plt.plot(estimates.traces)
plt.ylabel('Log-likelihood')
plt.xlabel('EM iteration');


For simulated data, we can also check how well our model recovered the original values for the electrode contributions and time distributions.

In [ ]:
plt.plot(true_pars.prod(axis=1), 'o', label='simulated',mec='k', mfc='w', )
plt.plot(estimates.parameters.prod(axis=1), 'x', label='estimated')#prod() is for a gamma only
plt.xticks(range(5),[1,2,3,4,5])
plt.legend()
plt.xlabel('Stage number')
plt.ylabel('Stage expected duration');

In [ ]:
for pc in range(init.n_dims):
    plt.plot(estimates.magnitudes.sel(component=pc), 'o', mec=colors[pc], mfc='w')
    plt.plot(true_magnitudes[:,pc], 'x', label=f'PC{pc}', color=colors[pc])
plt.xticks(range(4),[1,2,3,4])
plt.xlabel('Event number')
plt.ylabel('Contribution to the event')
plt.legend()

Even though the model is almost perfect on timing, it could improve on the PC contributions.

## Exploiting model estimates

In the end, what we are really interested in is the event probabilities, the probability of event occurring at each sample in every trial of every participants, as this allows us to do interesting follow-up analyses.

Let's check what these look like:

In [ ]:
estimates.eventprobs

This is trial x sample_in_trial x event matrix, where the values indicate the probablity of that event occurring at that sample.

We can represent this graphically, for example by plotting the average probability for each event at each time sample:

In [ ]:
plt.plot(estimates.eventprobs.mean('trial_x_participant'), label=[1,2,3,4])
plt.ylabel('P(event)')
plt.xlabel('Time (sample)')
plt.legend();

And we can even look at a single trial:

In [ ]:
plt.plot(estimates.eventprobs.sel(trial_x_participant=('S0',0)), label=[1,2,3,4])
plt.ylabel('P(event)')
plt.xlabel('Time (sample)');

And in the case of simulation compare that to the actual generating moment:

In [ ]:
estimated_times = init.compute_times(init, estimates, duration=True, mean=False, add_rt=True).T
plt.plot(estimates.eventprobs.sel(trial_x_participant=('S0',0)), label=[1,2,3,4], alpha=.5)
plt.ylabel('P(event)')
plt.xlabel('Time (sample)')
plt.vlines(np.cumsum(sim_source_times[0,:]), 0,.1, color=colors, ls='--')
plt.vlines(np.cumsum(estimated_times.sel(trial_x_participant=('S0',0))), 0,.1, color=colors, ls='-')
plt.legend();

Here the solid lines are the estimated times, while the dotted lines in indicate the simulated events. For this trial, the estimation procedure had trouble locating the third and fourth events.

If we select for all trials the most likely sample to contain a transition we can take a look at the recovery of our simulated times on all trials:

In [ ]:
fig, ax = plt.subplots(n_stages,2, figsize=(10,2*n_stages), dpi=100, sharex=True)
i = 0
ax[0,0].set_title('point-wise')
ax[0,1].set_title('Distribution')
ax[-1,0].set_xlabel(f'Simulated by-trial stage {i+1}')

for event in estimated_times:
    sns.regplot(x=sim_source_times[:,i].T, y=event, ax=ax[i,0], color=colors[i])
    ax[i,0].plot([np.min(event), np.max(event)], [np.min(event), np.max(event)],'--', color='k')
    ax[i,0].set_ylabel(f'Estimated by-trial stage {i+1}')
    sns.kdeplot(event, ax=ax[i,1], color='orange')
    sns.kdeplot(sim_source_times[:,i].T, ax=ax[i,1], color='k')
    i+= 1

plt.xlim(0,250)
plt.tight_layout();

Cleary, the last two stages are the hardest onces to recover.

# Exercise


Now based on the assumptions of an HMP model in terms of expected stage distribution and length of the event, and based on the implemented possibilities of stage distribution (see below) and event duration (in theory from 1 to +$\infty$ but in practice limited by your sampling frequency and length of your epochs), suggest an alternative HMP model and test the recovery of the original parameters (gamma with a shape of 2 and an event duration of 50ms)

In [ ]:
#Illustrating implemented distributions
#The supported families are the one displayed in the next list
#shape is a semi-continous parameter (0 to +infinity)

fig, ax = plt.subplots(2,3, sharex=True, sharey=True, figsize=(10,4))
plt.suptitle('Effect of different shape parameters for a given mean of 100')
distributions = [[['gamma',1],['gamma',2],['gamma',4]],
                 [['wald',.1],['wald',.5],['wald',1]],
                 [['weibull',2],['weibull',3],['weibull',4]],
                 [['lognormal',.25],['lognormal',.5],['lognormal',1]],
                 [['log-logistic',2],['log-logistic',4],['log-logistic',6]],
                 [['half-normal',1],['rayleigh',2],['maxwell-boltzmann',3]],]
i,j = 0,0
axes = [ax[0,0],ax[0,1],ax[0,2],ax[1,0], ax[1,1],ax[1,2]]
for family in distributions:
    for distribution in family:
        hmp.visu.plot_expected_distribution(distribution[0], 100, distribution[1], location=0, xmax=300, label=f's={distribution[1]}', ax=axes[j])
    axes[j].legend()
    axes[j].set_title(family[0][0])
    j += 1
    
plt.xlim(0,300)
axes[-1].set_title('chi')
plt.tight_layout();

Replace event_width or distribution or shape (or all three) in the HMP initialisation, and check how well the recovery works:

In [ ]:
init = hmp.models.hmp(data=hmp_data, epoch_data=eeg_data, sfreq=eeg_data.sfreq,
                      event_width=50, distribution='gamma', shape=2)

estimates = init.fit_single(n_events)
hmp.visu.plot_topo_timecourse(eeg_data, estimates, positions, init, magnify=1, sensors=True, times_to_display = np.mean(np.cumsum(sim_source_times,axis=1),axis=0))

In [ ]:
fig, ax = plt.subplots(n_stages,2, figsize=(10,2*n_stages), dpi=100, sharex=True)
i = 0
ax[0,0].set_title('point-wise')
ax[0,1].set_title('Distribution (orange = estimated)')
ax[-1,0].set_xlabel(f'Simulated by-trial stage {i+1}')

for event in init.compute_times(init, estimates, duration=True, mean=False, add_rt=True).T:
    sns.regplot(x=sim_source_times[:,i].T, y=event, ax=ax[i,0], color=colors[i])
    ax[i,0].plot([np.min(event), np.max(event)], [np.min(event), np.max(event)],'--', color='k')
    ax[i,0].set_ylabel(f'Estimated by-trial stage {i+1}')
    sns.kdeplot(event, ax=ax[i,1], color='orange')
    sns.kdeplot(sim_source_times[:,i].T, ax=ax[i,1], color='k')
    i+= 1

plt.xlim(0,250)
plt.tight_layout();